In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
## mount error 나는 경우 
#!fusermount -u gdrive
#!google-drive-ocamlfuse gdrive

In [3]:
ls

gdrive/  sample_data/


In [4]:
!pip install tensorboardcolab
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

tbc=TensorBoardColab()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://47442692.ngrok.io


In [0]:
import os
os.chdir("gdrive/My Drive/big_project")

In [6]:
ls

 cigarette_detection.ipynb                       det_add0.py    Mask_RCNN/
 cigarette_detection_plus_non_epochs_100.ipynb   det_check.py   __pycache__/
 cigarette_detection_plus_non.ipynb              det.py         Untitled0.ipynb
'cigarette_detection_plus_non.ipynb의 사본'      det_sub.py
 cigarette_detection_video.ipynb                 images/


In [0]:
# 한번 만 실행하기 
#!git clone https://www.github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')

In [8]:
debugging = True
from keras import backend as K
K.clear_session()

W0812 02:27:18.195715 139737951491968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0812 02:27:18.200211 139737951491968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0812 02:27:18.242118 139737951491968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [9]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import random
import collections
# for visualization
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
ROOT_IMAGE_DIR = os.path.abspath("../images/")

# Path to the dataset (note this is a shared images directory)
dataset_path = os.path.join(ROOT_IMAGE_DIR)

models_dir = os.path.join(ROOT_DIR,  "Mask_RCNN/models")

print("base dataset dir:", dataset_path)
print("base models dir:", models_dir)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.model import log

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(models_dir, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")


base dataset dir: /content/gdrive/My Drive/big_project/images
base models dir: /content/gdrive/My Drive/big_project/Mask_RCNN/models


In [0]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

%matplotlib inline

In [0]:
import det as det

In [0]:
# The imgaug library is pretty flexible and make different types of augmentation possible. 
# The deterministic setting is used because any spatial changes to the image must also be 
# done to the mask. There are also some augmentors that are unsafe to apply. From the mrcnn
# library: 
# Augmentors that are safe to apply to masks: 
# ["Sequential", "SomeOf", "OneOf", "Sometimes","Fliplr", 
# "Flipud", "CropAndPad", "Affine", "PiecewiseAffine"]
# Affine, has settings that are unsafe, so always
# test your augmentation on masks

import imgaug as ia
from imgaug import augmenters as iaa

ia.seed(1)

# http://imgaug.readthedocs.io/en/latest/source/augmenters.html#sequential
seq_of_aug = iaa.Sequential([
    iaa.Crop(percent=(0, 0.1)), # random crops
    
    # horizontally flip 50% of the images
    iaa.Fliplr(0.5), 

    # Gaussian blur to 50% of the images
    # with random sigma between 0 and 0.5.
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    
    # Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    
    # Apply affine transformations to each image.
    # Scale/zoom them from 90% 5o 110%
    # Translate/move them, rotate them
    # Shear them slightly -2 to 2 degrees.
    iaa.Affine(
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-5, 5),
        shear=(-2, 2)
    )
], random_order=True) # apply augmenters in random order

In [13]:
VIDEO_DIR = ROOT_IMAGE_DIR
VIDEO_DIR

'/content/gdrive/My Drive/big_project/images'

In [14]:
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "video_output")
VIDEO_SAVE_DIR

'/content/gdrive/My Drive/big_project/images/video_output'

In [0]:
class InferenceConfig(det.DetConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
inf_config = InferenceConfig('cigarette', ['cigarette'])

In [16]:
inf_config.ACTIVATION = 'leakyrelu'
inf_config.CLASS_NAMES.insert(0, 'BG')
inf_config.DETECTION_MIN_CONFIDENCE=0.9
inf_config.display()


Configurations:
ACTIVATION                     leakyrelu
ALL_CLASS_NAMES                ['BG', 'cigarette']
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
CLASS_NAMES                    ['BG', 'cigarette']
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_

In [0]:
import cv2
import numpy as np


def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

In [0]:

from PIL import Image, ImageDraw, ImageFont

def display_instances(image, boxes, masks, class_ids, class_names,
                      scores, image_name, save_dir, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=True,
                      colors=None, captions=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """
    
    N = boxes.shape[0]
    colors = colors or random_colors(N)

    if not N:
        print("\n*** No instances in image %s to draw *** \n" % (image_name))
        masked_image = image.astype(np.uint8).copy()
        cv2.imwrite(os.path.join(save_dir, '%s' % (image_name)), masked_image) 
        return
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    useful_mask_indices = []
    
    for i in range(N):
        # Generate random colors
        colors = colors or random_colors(N)
        
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        useful_mask_indices.append(i)

    masked_image = image.astype(np.uint8).copy()
    for index, value in enumerate(useful_mask_indices):
        masked_image = apply_mask(masked_image, masks[:, :, value], colors[index])
        
    masked_image = Image.fromarray(masked_image)
    draw = ImageDraw.Draw(masked_image)
    colors = np.array(colors).astype(int) * 255
    
    for index, value in enumerate(useful_mask_indices):
        class_id = class_ids[value]
        score = scores[value]
        label = class_names[class_id]

        y1, x1, y2, x2 = boxes[value]
        color = tuple(colors[index])
        draw.rectangle((x1, y1, x2, y2), outline=color)

        # Label
        #font = ImageFont.truetype('/Library/Fonts/Arial.ttf', 15)
        draw.text((x1, y1), "%s %f" % (label, score), (255, 255, 255))

        masked_image.save(os.path.join(save_dir, '%s' % (image_name)))

In [19]:
inf_model = modellib.MaskRCNN(mode="inference", 
                              config=inf_config,
                              model_dir=models_dir)

inf_config.display()

# Find last trained weights
#weights_path = inf_model.find_last()[1]

# When testing use the latest, for demo use stable model

#weights_path = os.path.join(models_dir+'fish20190217T2122', "mask_rcnn_fish_0070.h5")
weights_path = os.path.join(models_dir+'/cigarette20190811T1717', "mask_rcnn_cigarette_0021.h5")
# weights_path = os.path.join(models_dir, "wolf20180516T0136_mask_rcnn_wolf_0075.h5")


print("Using latest weights: ", weights_path)

inf_model.load_weights(weights_path, by_name=True)

W0812 02:27:21.388193 139737951491968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 02:27:21.398293 139737951491968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0812 02:27:21.434772 139737951491968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1919: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0812 02:27:21.438255 139737951491968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0812 02:27:24.071057 139737951491968 deprecation


Configurations:
ACTIVATION                     leakyrelu
ALL_CLASS_NAMES                ['BG', 'cigarette']
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
CLASS_NAMES                    ['BG', 'cigarette']
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_

In [20]:
print(VIDEO_DIR)

/content/gdrive/My Drive/big_project/images


In [21]:
import cv2
import colorsys
from skimage.measure import find_contours

print(os.path.join(VIDEO_DIR, 'input.mP4'))
capture = cv2.VideoCapture(os.path.join(VIDEO_DIR, 'input.mp4')) 
print(capture)
batch_size = 1
frame_count = 0
while True:
    ret, frame = capture.read()
    #print(ret, frame)
    # Bail out when the video file ends
    if not ret:
        break        
    # Save each frame of the video to a list
    frame_count +=  1
    print(frame_count)
    frames = []
    frames.append(frame)
    if len(frames) == batch_size:
        results = inf_model.detect(frames, verbose=0)
        for i, item in enumerate(zip(frames, results)):
            frame = item[0]
            r = item[1]
            name = '{0}.jpg'.format(frame_count + i - batch_size)
            frame = display_instances(
                frame, r['rois'], r['masks'], r['class_ids'], inf_config.CLASS_NAMES, r['scores'],
                name, VIDEO_SAVE_DIR, 
            )
        # Clear the frames array to start the next batch
        frames = []

/content/gdrive/My Drive/big_project/images/input.mP4
<VideoCapture 0x7f16e3e96430>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166

*** No instances in image 165.jpg to draw *** 

167

*** No instances in image 166.jpg to draw *** 

168

*** No instances in image 167.jpg to draw *** 

169
170

*** No instances in image 169.jpg to draw *** 

171

*** No instances in image 170.jpg to draw *** 

172

*** No instances in image 171.jpg to draw *** 

173

*** No instances in image 172.jpg to 

In [23]:
import glob
# Get all image file paths to a list.
images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))
# Sort the images by name index.
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-4]))
images

['/content/gdrive/My Drive/big_project/images/video_output/0.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/1.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/2.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/3.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/4.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/5.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/6.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/7.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/8.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/9.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/10.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/11.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/12.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output/13.jpg',
 '/content/gdrive/My Drive/big_project/images/video_output

In [24]:
video = cv2.VideoCapture(os.path.join(VIDEO_DIR, 'input.mp4'));

# Find OpenCV version
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if int(major_ver)  < 3 :
    fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
else :
    fps = video.get(cv2.CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

video.release();

Frames per second using video.get(cv2.CAP_PROP_FPS) : 23.976023976023978


In [28]:
def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    print('len(images):', len(images))
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

import glob
import os

# Directory of images to run detection on
#ROOT_DIR = os.getcwd()
#VIDEO_DIR = os.path.join(ROOT_DIR, "videos")
#VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "save")
images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))
# Sort the images by integer index
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-4]))

outvid = os.path.join(VIDEO_DIR, "result.mp4")
print(outvid)

/content/gdrive/My Drive/big_project/images/result.mp4


In [29]:
make_video(outvid, images, fps)

len(images): 720


<VideoWriter 0x7f1689d5e7d0>